<a href="https://colab.research.google.com/github/Alfaalputra/machine-learning/blob/main/Asosiasi_Tuberkulosis_Anak_FP_Growth_Library.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Preprocessing Data

In [ ]:
import pandas as pd
import numpy as np
import itertools
import collections
import math

from scipy.sparse import csr_matrix
from itertools import combinations

In [ ]:
dataset = pd.read_csv('/content/gdrive/MyDrive/dataset TB Anak/tb.csv')
dataset.head()

,Timestamp,Tanggal Lahir,Umur,Tinggi badan (dalam cm),Berat badan (dalam kg),Jenis Kelamin,Alamat (Kelurahan),Alamat (Kecamatan),Alamat (Kab/Kota),Alamat (mohon sertakan nama kelurahan dan kecamatan),...,Apakah anak mengkonsumsi ASI secara eksklusif? (ASI Eksklusif adalah pemberian ASI tanpa makanan/minuman (susu formula) tambahan hingga berusia 6 bulan),Apakah ada riwayat penyakit tuberkulosis dalam orang serumah?,Apakah ada riwayat penyakit diabetes dalam keluarga (orang tua)?,"Apakah ada riwayat penyakit lainnya selain tuberkulosis, diabetes dalam orang serumah?","Jika ada, penyakit apa saja?",Apakah ada yang pernah atau sedang mengkonsumsi obat tuberkulosis dalam orang serumah?,Berapa luas rumah tempat anak tinggal?,Berapa jumlah kamar tidur dalam rumah?,Berapa jumlah orang yang tinggal dalam satu rumah?,Bagaimana sistem ventilasi di rumah Anda?
0,07/10/2021 12:03:36,20/11/2014,6 Tahun,125.0,19.0,Laki - laki,Noeha,Nuha,Sorowako,"Kec. Nuha, Sorowako",...,Ya,Tidak,Tidak,Ya,Hipertensi,Tidak,> 120 m^2,4,8,Setiap ruangan terdapat ventilasi
1,07/10/2021 14:48:47,18/04/2011,10 Tahun,120.0,30.0,Perempuan,Parang Tambung,Tamalate,Makassar,Dg.tata 1 perumahan tata satu indah blok 1b,...,Ya,Tidak,Tidak,Ya,Tidak Ada,Tidak,> 120 m^2,5,8,Setiap ruangan terdapat ventilasi
2,10/10/2021 21:54:49,04/05/2018,3 Tahun 5 Bulan,104.0,17.0,Laki - laki,Kalebajeng,Bajeng,Gowa,Perum bajeng permai blok u no 4. Kel kalebajen...,...,Tidak,Tidak,Tidak,Tidak,Tidak Ada,Tidak,54 - 120 m^2,2,5,Setiap ruangan terdapat ventilasi
3,11/10/2021 17:03:37,20/05/2013,8 Tahun,120.0,23.0,Perempuan,Bone,Masamba,Luwu Utara,"Jl.Mujahidin Kel.Bone Kec.Masamba,Luwu Utara",...,Ya,Tidak,Tidak,Tidak,Tidak Ada,Tidak,54 - 120 m^2,4,5,Setiap ruangan terdapat ventilasi
4,14/10/2021 16:39:34,11/02/2019,2 Tahun 8 Bulan,90.0,14.0,Perempuan,Borong,Manggala,Makassar,"Perm. Graha Indah Family, Kel. Borong, Kec. Ma...",...,Tidak,Tidak,Tidak,Ya,Bronkitis,Tidak,54 - 120 m^2,4,5,Ada ventilasi namun kadang terbuka/tertutup


In [ ]:
locations = []
coord = set(dataset['Alamat (Kab/Kota)'] + ',Indonesia')
geolocator = Nominatim(user_agent="arcgis")
for kota in coord:
  location = geolocator.geocode(kota, timeout=None)
  locations.append([location.address, location.latitude, location.longitude])

dataKota = pd.DataFrame(locations, columns=['address', 'lat', 'long'])
dataKota

,address,lat,long
0,"Ternate, Maluku Utara, Indonesia",0.785234,127.383175
1,"Pangkajene dan Kepulauan, Sulawesi Selatan, In...",-4.754450,119.589311
2,"Bantaeng, Sulawesi Selatan, Indonesia",-5.470945,119.987168
3,"Takalar, Canrego, Sulawesi Selatan, Indonesia",-5.448702,119.467822
4,"Maluku Utara, Indonesia",0.630122,127.972022
5,"Bekasi, Jawa Barat, 17144, Indonesia",-6.234986,106.994544
6,"Luwu Utara, Sulawesi Selatan, Indonesia",-2.595126,120.256254
7,"Bandara Sorowako, Jembatan Merah, Nikkel, Sula...",-2.529616,121.355816
8,"Barru, Sulawesi Selatan, Indonesia",-4.433734,119.677827
9,"Sinjai, Sulawesi Selatan, Indonesia",-5.200667,120.144187


In [ ]:
coord

{'Bantaeng,Indonesia',
 'Barru,Indonesia',
 'Bekasi,Indonesia',
 'Gowa,Indonesia',
 'Luwu Utara,Indonesia',
 'Makassar,Indonesia',
 'Maluku Utara,Indonesia',
 'Maros,Indonesia',
 'Pangkajene dan Kepulauan,Indonesia',
 'Polewali Mandar,Indonesia',
 'Sinjai,Indonesia',
 'Sorowako,Indonesia',
 'Takalar,Indonesia',
 'Ternate,Indonesia'}

In [ ]:
def coordinateKota(df):
    coord = df['Alamat (Kab/Kota)']
    geolocator = Nominatim(user_agent="arcgis")
    kota = []
    location = []
    for i, j in enumerate(coord):
        kota_item = ','.join(str(x) for x in j)
        kota.append(kota_item)
        location = geolocator.geocode(j, timeout=None)
        if location != None:
            location.append([location.address, location.latitude, location.longitude])

    locations = pd.DataFrame(location, columns=['address', 'lat', 'long'])
    return locations

In [ ]:
from geopy.geocoders import Nominatim

In [ ]:
a = coordinateKota(dataset)

AttributeError: ignored

In [ ]:
df = df.rename(columns={
    'Alamat (mohon sertakan nama kelurahan dan kecamatan)':'Alamat lengkap',
    'Apakah anak pernah atau sedang dalam pengobatan tuberkulosis?':'pernah/sedang TB',
    'Apakah anak pernah mengalami penyakit diabetes?':'riwayat diabetes anak',
    'Apakah anak telah menerima imunisasi BCG (Bacillus Calmette-Guérin, imunisasi untuk mencegah penyakit TB)?':'riwayat vaksin BCG',
    'Apakah anak pernah di opname sebelumnya?':'riwayat opname',
    'Jika pernah, anak diopname karena penyakit apa saja?':'daftar penyakit opname' ,
    'Apakah anak mengkonsumsi ASI secara eksklusif? (ASI Eksklusif adalah pemberian ASI tanpa makanan/minuman (susu formula) tambahan hingga berusia 6 bulan)':'ASI eksklusif',
    'Apakah ada riwayat penyakit tuberkulosis dalam orang serumah?':'riwayat TB orang serumah',
    'Apakah ada riwayat penyakit diabetes dalam keluarga (orang tua)?':'riwayat diabetes keluarga',
    'Apakah ada riwayat penyakit lainnya selain tuberkulosis, diabetes dalam orang  serumah?':'riwayat penyakit lain orang serumah',
    'Jika ada, penyakit apa saja?':'daftar penyakit lain orang serumah',
    'Berapa luas rumah tempat anak tinggal?':'luas rumah',
    'Berapa jumlah kamar tidur dalam rumah?':'jumlah kamar tidur',
    'Berapa jumlah orang yang tinggal dalam satu rumah?':'jumlah orang dalam rumah',
    'Bagaimana sistem ventilasi di rumah Anda? ':'sistem ventilasi' })

In [ ]:
cp.drop(['Timestamp',
    'Tanggal Lahir',
    'Alamat lengkap',
    'pernah/sedang TB',
    'riwayat opname',
    'riwayat penyakit lain orang serumah',
    'Apakah ada yang pernah atau sedang mengkonsumsi obat tuberkulosis dalam orang serumah?'],
    axis = 'columns', inplace = True)
cp.head()

,Umur,Tinggi badan (dalam cm),Berat badan (dalam kg),Jenis Kelamin,Alamat (Kelurahan),Alamat (Kecamatan),Alamat (Kab/Kota),Pekerjaan Ayah,Pekerjaan Ibu,Pendapatan Orang Tua,riwayat diabetes anak,riwayat vaksin BCG,daftar penyakit opname,ASI eksklusif,riwayat TB orang serumah,riwayat diabetes keluarga,daftar penyakit lain orang serumah,luas rumah,jumlah kamar tidur,jumlah orang dalam rumah,sistem ventilasi
0,6 Tahun,125.0,19.0,Laki - laki,Noeha,Nuha,Sorowako,Pegawai Swasta,Ibu Rumah Tangga,> 10.000.000,Tidak,Ya,Diare,Ya,Tidak,Tidak,Hipertensi,> 120 m^2,4,8,Setiap ruangan terdapat ventilasi
1,10 Tahun,120.0,30.0,Perempuan,Parang Tambung,Tamalate,Makassar,Wiraswasta,ASN,2.000.000 - 5.000.000,Tidak,Tidak,Brongkhitis,Ya,Tidak,Tidak,Tidak Ada,> 120 m^2,5,8,Setiap ruangan terdapat ventilasi
2,3 Tahun 5 Bulan,104.0,17.0,Laki - laki,Kalebajeng,Bajeng,Gowa,Wiraswasta,Ibu rumah tangga,2.000.000 - 5.000.000,Tidak,Ya,"Tipoid, Diare, Bronkopneumonia, Demam",Tidak,Tidak,Tidak,Tidak Ada,54 - 120 m^2,2,5,Setiap ruangan terdapat ventilasi
3,8 Tahun,120.0,23.0,Perempuan,Bone,Masamba,Luwu Utara,ASN,ASN,5.000.000 - 10.000.000,Tidak,Tidak,Tidak Pernah,Ya,Tidak,Tidak,Tidak Ada,54 - 120 m^2,4,5,Setiap ruangan terdapat ventilasi
4,2 Tahun 8 Bulan,90.0,14.0,Perempuan,Borong,Manggala,Makassar,ASN,Dokter Gigi,> 10.000.000,Tidak,Ya,Tuberkulosis,Tidak,Tidak,Tidak,Bronkitis,54 - 120 m^2,4,5,Ada ventilasi namun kadang terbuka/tertutup


In [ ]:
new = cp['Umur'].str.split(r'(\d+)', expand = True)
new = new.replace([None], '0')

cp['Tahun']= new[1]
cp['Bulan'] = new[3]

cp.drop(['Umur'], axis = 'columns', inplace = True)
cp

,Tinggi badan (dalam cm),Berat badan (dalam kg),Jenis Kelamin,Alamat (Kelurahan),Alamat (Kecamatan),Alamat (Kab/Kota),Pekerjaan Ayah,Pekerjaan Ibu,Pendapatan Orang Tua,riwayat diabetes anak,riwayat vaksin BCG,daftar penyakit opname,ASI eksklusif,riwayat TB orang serumah,riwayat diabetes keluarga,daftar penyakit lain orang serumah,luas rumah,jumlah kamar tidur,jumlah orang dalam rumah,sistem ventilasi,Tahun,Bulan
0,125.0,19.00,Laki - laki,Noeha,Nuha,Sorowako,Pegawai Swasta,Ibu Rumah Tangga,> 10.000.000,Tidak,Ya,Diare,Ya,Tidak,Tidak,Hipertensi,> 120 m^2,4,8,Setiap ruangan terdapat ventilasi,6,0
1,120.0,30.00,Perempuan,Parang Tambung,Tamalate,Makassar,Wiraswasta,ASN,2.000.000 - 5.000.000,Tidak,Tidak,Brongkhitis,Ya,Tidak,Tidak,Tidak Ada,> 120 m^2,5,8,Setiap ruangan terdapat ventilasi,10,0
2,104.0,17.00,Laki - laki,Kalebajeng,Bajeng,Gowa,Wiraswasta,Ibu rumah tangga,2.000.000 - 5.000.000,Tidak,Ya,"Tipoid, Diare, Bronkopneumonia, Demam",Tidak,Tidak,Tidak,Tidak Ada,54 - 120 m^2,2,5,Setiap ruangan terdapat ventilasi,3,5
3,120.0,23.00,Perempuan,Bone,Masamba,Luwu Utara,ASN,ASN,5.000.000 - 10.000.000,Tidak,Tidak,Tidak Pernah,Ya,Tidak,Tidak,Tidak Ada,54 - 120 m^2,4,5,Setiap ruangan terdapat ventilasi,8,0
4,90.0,14.00,Perempuan,Borong,Manggala,Makassar,ASN,Dokter Gigi,> 10.000.000,Tidak,Ya,Tuberkulosis,Tidak,Tidak,Tidak,Bronkitis,54 - 120 m^2,4,5,Ada ventilasi namun kadang terbuka/tertutup,2,8
5,82.0,8.40,Laki - laki,Sengka,Bontonompo Selatan,Gowa,Perawat,Ibu Rumah tangga,2.000.000 - 5.000.000,Tidak,Ya,Tidak Pernah,Ya,Tidak,Ya,Tidak ada,36 - 54 m^2,3,4,Ada ventilasi namun kadang terbuka/tertutup,1,3
6,80.0,11.00,Laki - laki,Sudiang Raya,Biring Kanaya,Makassar,Pegawai swasta,Ibu Rumah tangga,5.000.000 - 10.000.000,Tidak,Ya,Tidak Pernah,Ya,Ya,Tidak,"Asma, Eczema, Asam Urat, Hipertensi, Kolesterol",36 - 54 m^2,3,3,Ada ventilasi namun selalu tertutup,3,0
7,84.0,10.00,Laki - laki,Palakka,Barru,Barru,ASN,ASN,2.000.000 - 5.000.000,Tidak,Tidak,Pneumonia,Tidak,Tidak,Tidak,"Hipertensi, Kolesterol",54 - 120 m^2,3,5,Ada ventilasi namun kadang terbuka/tertutup,2,8
8,125.0,27.00,Perempuan,Malimongan Baru,Bontoala,Makassar,Wiraswasta,Wiraswasta,> 10.000.000,Tidak,Ya,"Demam, Radang Tenggorokan",Ya,Tidak,Tidak,Hipertensi,> 120 m^2,4,7,Setiap ruangan terdapat ventilasi,7,0
9,80.0,10.50,Laki - laki,Tamalanrea Jaya,Tamalanrea,Makassar,Pegawai Swasta,Tidak Ada,2.000.000 - 5.000.000,Tidak,Ya,Tidak Pernah,Ya,Tidak,Ya,Tidak Ada,54 - 120 m^2,5,8,Setiap ruangan terdapat ventilasi,1,7


In [ ]:
cp.replace(np.nan, 'Tidak Ada', inplace = True)
cp

,Tinggi badan (dalam cm),Berat badan (dalam kg),Jenis Kelamin,Alamat (Kelurahan),Alamat (Kecamatan),Alamat (Kab/Kota),Pekerjaan Ayah,Pekerjaan Ibu,Pendapatan Orang Tua,riwayat diabetes anak,riwayat vaksin BCG,daftar penyakit opname,ASI eksklusif,riwayat TB orang serumah,riwayat diabetes keluarga,daftar penyakit lain orang serumah,luas rumah,jumlah kamar tidur,jumlah orang dalam rumah,sistem ventilasi,Tahun,Bulan
0,125.0,19.00,Laki - laki,Noeha,Nuha,Sorowako,Pegawai Swasta,Ibu Rumah Tangga,> 10.000.000,Tidak,Ya,Diare,Ya,Tidak,Tidak,Hipertensi,> 120 m^2,4,8,Setiap ruangan terdapat ventilasi,6,0
1,120.0,30.00,Perempuan,Parang Tambung,Tamalate,Makassar,Wiraswasta,ASN,2.000.000 - 5.000.000,Tidak,Tidak,Brongkhitis,Ya,Tidak,Tidak,Tidak Ada,> 120 m^2,5,8,Setiap ruangan terdapat ventilasi,10,0
2,104.0,17.00,Laki - laki,Kalebajeng,Bajeng,Gowa,Wiraswasta,Ibu rumah tangga,2.000.000 - 5.000.000,Tidak,Ya,"Tipoid, Diare, Bronkopneumonia, Demam",Tidak,Tidak,Tidak,Tidak Ada,54 - 120 m^2,2,5,Setiap ruangan terdapat ventilasi,3,5
3,120.0,23.00,Perempuan,Bone,Masamba,Luwu Utara,ASN,ASN,5.000.000 - 10.000.000,Tidak,Tidak,Tidak Pernah,Ya,Tidak,Tidak,Tidak Ada,54 - 120 m^2,4,5,Setiap ruangan terdapat ventilasi,8,0
4,90.0,14.00,Perempuan,Borong,Manggala,Makassar,ASN,Dokter Gigi,> 10.000.000,Tidak,Ya,Tuberkulosis,Tidak,Tidak,Tidak,Bronkitis,54 - 120 m^2,4,5,Ada ventilasi namun kadang terbuka/tertutup,2,8
5,82.0,8.40,Laki - laki,Sengka,Bontonompo Selatan,Gowa,Perawat,Ibu Rumah tangga,2.000.000 - 5.000.000,Tidak,Ya,Tidak Pernah,Ya,Tidak,Ya,Tidak ada,36 - 54 m^2,3,4,Ada ventilasi namun kadang terbuka/tertutup,1,3
6,80.0,11.00,Laki - laki,Sudiang Raya,Biring Kanaya,Makassar,Pegawai swasta,Ibu Rumah tangga,5.000.000 - 10.000.000,Tidak,Ya,Tidak Pernah,Ya,Ya,Tidak,"Asma, Eczema, Asam Urat, Hipertensi, Kolesterol",36 - 54 m^2,3,3,Ada ventilasi namun selalu tertutup,3,0
7,84.0,10.00,Laki - laki,Palakka,Barru,Barru,ASN,ASN,2.000.000 - 5.000.000,Tidak,Tidak,Pneumonia,Tidak,Tidak,Tidak,"Hipertensi, Kolesterol",54 - 120 m^2,3,5,Ada ventilasi namun kadang terbuka/tertutup,2,8
8,125.0,27.00,Perempuan,Malimongan Baru,Bontoala,Makassar,Wiraswasta,Wiraswasta,> 10.000.000,Tidak,Ya,"Demam, Radang Tenggorokan",Ya,Tidak,Tidak,Hipertensi,> 120 m^2,4,7,Setiap ruangan terdapat ventilasi,7,0
9,80.0,10.50,Laki - laki,Tamalanrea Jaya,Tamalanrea,Makassar,Pegawai Swasta,Tidak Ada,2.000.000 - 5.000.000,Tidak,Ya,Tidak Pernah,Ya,Tidak,Ya,Tidak Ada,54 - 120 m^2,5,8,Setiap ruangan terdapat ventilasi,1,7


In [ ]:
cp.dtypes

Tinggi badan (dalam cm)               float64
Berat badan (dalam kg)                float64
Jenis Kelamin                          object
Alamat (Kelurahan)                     object
Alamat (Kecamatan)                     object
Alamat (Kab/Kota)                      object
Pekerjaan Ayah                         object
Pekerjaan Ibu                          object
Pendapatan Orang Tua                   object
riwayat diabetes anak                  object
riwayat vaksin BCG                     object
daftar penyakit opname                 object
ASI eksklusif                          object
riwayat TB orang serumah               object
riwayat diabetes keluarga              object
daftar penyakit lain orang serumah     object
luas rumah                             object
jumlah kamar tidur                      int64
jumlah orang dalam rumah                int64
sistem ventilasi                       object
Tahun                                  object
Bulan                             

In [ ]:
cp['jumlah kamar tidur'] = cp['jumlah kamar tidur'].astype(str)
cp['jumlah orang dalam rumah'] = cp['jumlah orang dalam rumah'].astype(str)

In [ ]:
# cp['pernah/sedang TB'] = cp['pernah/sedang TB'] + ' pernah/sedang TB'
cp['Alamat (Kelurahan)'] = 'Kelurahan ' + cp['Alamat (Kelurahan)']
cp['Alamat (Kecamatan)'] = 'Kecamatan ' + cp['Alamat (Kecamatan)']
cp['Pekerjaan Ayah'] = 'Ayah ' + cp['Pekerjaan Ayah']
cp['Pekerjaan Ibu'] = 'Ibu ' + cp['Pekerjaan Ibu']
cp['jumlah kamar tidur'] = cp['jumlah kamar tidur'] + ' Kamar'
cp['jumlah orang dalam rumah'] = cp['jumlah orang dalam rumah'] + ' Orang'
cp['riwayat diabetes anak'] =  cp['riwayat diabetes anak'] + ' pernah diabetes'
cp['riwayat vaksin BCG'] = cp['riwayat vaksin BCG'] + ' Anak Telah BCG'
cp['ASI eksklusif'] = cp['ASI eksklusif'] + ' ASI Ekslusif'
cp['riwayat TB orang serumah'] = cp['riwayat TB orang serumah'] + ' ada riwayat TB orang serumah'
cp['riwayat diabetes keluarga'] = cp['riwayat diabetes keluarga'] + ' ada riwayat penyakit diabetes (orang tua)'
cp['daftar penyakit opname'] = cp['daftar penyakit opname'] + ' (Anak)'
cp['daftar penyakit lain orang serumah'] = cp['daftar penyakit lain orang serumah'] + ' (orang serumah)'

cp.head()

,Tinggi badan (dalam cm),Berat badan (dalam kg),Jenis Kelamin,Alamat (Kelurahan),Alamat (Kecamatan),Alamat (Kab/Kota),Pekerjaan Ayah,Pekerjaan Ibu,Pendapatan Orang Tua,riwayat diabetes anak,riwayat vaksin BCG,daftar penyakit opname,ASI eksklusif,riwayat TB orang serumah,riwayat diabetes keluarga,daftar penyakit lain orang serumah,luas rumah,jumlah kamar tidur,jumlah orang dalam rumah,sistem ventilasi,Tahun,Bulan
0,125.0,19.0,Laki - laki,Kelurahan Noeha,Kecamatan Nuha,Sorowako,Ayah Pegawai Swasta,Ibu Ibu Rumah Tangga,> 10.000.000,Tidak pernah diabetes,Ya Anak Telah BCG,Diare (Anak),Ya ASI Ekslusif,Tidak ada riwayat TB orang serumah,Tidak ada riwayat penyakit diabetes (orang tua),Hipertensi (orang serumah),> 120 m^2,4 Kamar,8 Orang,Setiap ruangan terdapat ventilasi,6,0
1,120.0,30.0,Perempuan,Kelurahan Parang Tambung,Kecamatan Tamalate,Makassar,Ayah Wiraswasta,Ibu ASN,2.000.000 - 5.000.000,Tidak pernah diabetes,Tidak Anak Telah BCG,Brongkhitis (Anak),Ya ASI Ekslusif,Tidak ada riwayat TB orang serumah,Tidak ada riwayat penyakit diabetes (orang tua),Tidak Ada (orang serumah),> 120 m^2,5 Kamar,8 Orang,Setiap ruangan terdapat ventilasi,10,0
2,104.0,17.0,Laki - laki,Kelurahan Kalebajeng,Kecamatan Bajeng,Gowa,Ayah Wiraswasta,Ibu Ibu rumah tangga,2.000.000 - 5.000.000,Tidak pernah diabetes,Ya Anak Telah BCG,"Tipoid, Diare, Bronkopneumonia, Demam (Anak)",Tidak ASI Ekslusif,Tidak ada riwayat TB orang serumah,Tidak ada riwayat penyakit diabetes (orang tua),Tidak Ada (orang serumah),54 - 120 m^2,2 Kamar,5 Orang,Setiap ruangan terdapat ventilasi,3,5
3,120.0,23.0,Perempuan,Kelurahan Bone,Kecamatan Masamba,Luwu Utara,Ayah ASN,Ibu ASN,5.000.000 - 10.000.000,Tidak pernah diabetes,Tidak Anak Telah BCG,Tidak Pernah (Anak),Ya ASI Ekslusif,Tidak ada riwayat TB orang serumah,Tidak ada riwayat penyakit diabetes (orang tua),Tidak Ada (orang serumah),54 - 120 m^2,4 Kamar,5 Orang,Setiap ruangan terdapat ventilasi,8,0
4,90.0,14.0,Perempuan,Kelurahan Borong,Kecamatan Manggala,Makassar,Ayah ASN,Ibu Dokter Gigi,> 10.000.000,Tidak pernah diabetes,Ya Anak Telah BCG,Tuberkulosis (Anak),Tidak ASI Ekslusif,Tidak ada riwayat TB orang serumah,Tidak ada riwayat penyakit diabetes (orang tua),Bronkitis (orang serumah),54 - 120 m^2,4 Kamar,5 Orang,Ada ventilasi namun kadang terbuka/tertutup,2,8


In [ ]:
tinggi_meter = cp['Tinggi badan (dalam cm)']/100
cp['IMT'] = cp['Berat badan (dalam kg)']/ (pow(tinggi_meter, 2))

cp

,Tinggi badan (dalam cm),Berat badan (dalam kg),Jenis Kelamin,Alamat (Kelurahan),Alamat (Kecamatan),Alamat (Kab/Kota),Pekerjaan Ayah,Pekerjaan Ibu,Pendapatan Orang Tua,riwayat diabetes anak,riwayat vaksin BCG,daftar penyakit opname,ASI eksklusif,riwayat TB orang serumah,riwayat diabetes keluarga,daftar penyakit lain orang serumah,luas rumah,jumlah kamar tidur,jumlah orang dalam rumah,sistem ventilasi,Tahun,Bulan,IMT
0,125.0,19.00,Laki - laki,Kelurahan Noeha,Kecamatan Nuha,Sorowako,Ayah Pegawai Swasta,Ibu Ibu Rumah Tangga,> 10.000.000,Tidak pernah diabetes,Ya Anak Telah BCG,Diare (Anak),Ya ASI Ekslusif,Tidak ada riwayat TB orang serumah,Tidak ada riwayat penyakit diabetes (orang tua),Hipertensi (orang serumah),> 120 m^2,4 Kamar,8 Orang,Setiap ruangan terdapat ventilasi,6,0,12.160000
1,120.0,30.00,Perempuan,Kelurahan Parang Tambung,Kecamatan Tamalate,Makassar,Ayah Wiraswasta,Ibu ASN,2.000.000 - 5.000.000,Tidak pernah diabetes,Tidak Anak Telah BCG,Brongkhitis (Anak),Ya ASI Ekslusif,Tidak ada riwayat TB orang serumah,Tidak ada riwayat penyakit diabetes (orang tua),Tidak Ada (orang serumah),> 120 m^2,5 Kamar,8 Orang,Setiap ruangan terdapat ventilasi,10,0,20.833333
2,104.0,17.00,Laki - laki,Kelurahan Kalebajeng,Kecamatan Bajeng,Gowa,Ayah Wiraswasta,Ibu Ibu rumah tangga,2.000.000 - 5.000.000,Tidak pernah diabetes,Ya Anak Telah BCG,"Tipoid, Diare, Bronkopneumonia, Demam (Anak)",Tidak ASI Ekslusif,Tidak ada riwayat TB orang serumah,Tidak ada riwayat penyakit diabetes (orang tua),Tidak Ada (orang serumah),54 - 120 m^2,2 Kamar,5 Orang,Setiap ruangan terdapat ventilasi,3,5,15.717456
3,120.0,23.00,Perempuan,Kelurahan Bone,Kecamatan Masamba,Luwu Utara,Ayah ASN,Ibu ASN,5.000.000 - 10.000.000,Tidak pernah diabetes,Tidak Anak Telah BCG,Tidak Pernah (Anak),Ya ASI Ekslusif,Tidak ada riwayat TB orang serumah,Tidak ada riwayat penyakit diabetes (orang tua),Tidak Ada (orang serumah),54 - 120 m^2,4 Kamar,5 Orang,Setiap ruangan terdapat ventilasi,8,0,15.972222
4,90.0,14.00,Perempuan,Kelurahan Borong,Kecamatan Manggala,Makassar,Ayah ASN,Ibu Dokter Gigi,> 10.000.000,Tidak pernah diabetes,Ya Anak Telah BCG,Tuberkulosis (Anak),Tidak ASI Ekslusif,Tidak ada riwayat TB orang serumah,Tidak ada riwayat penyakit diabetes (orang tua),Bronkitis (orang serumah),54 - 120 m^2,4 Kamar,5 Orang,Ada ventilasi namun kadang terbuka/tertutup,2,8,17.283951
5,82.0,8.40,Laki - laki,Kelurahan Sengka,Kecamatan Bontonompo Selatan,Gowa,Ayah Perawat,Ibu Ibu Rumah tangga,2.000.000 - 5.000.000,Tidak pernah diabetes,Ya Anak Telah BCG,Tidak Pernah (Anak),Ya ASI Ekslusif,Tidak ada riwayat TB orang serumah,Ya ada riwayat penyakit diabetes (orang tua),Tidak ada (orang serumah),36 - 54 m^2,3 Kamar,4 Orang,Ada ventilasi namun kadang terbuka/tertutup,1,3,12.492564
6,80.0,11.00,Laki - laki,Kelurahan Sudiang Raya,Kecamatan Biring Kanaya,Makassar,Ayah Pegawai swasta,Ibu Ibu Rumah tangga,5.000.000 - 10.000.000,Tidak pernah diabetes,Ya Anak Telah BCG,Tidak Pernah (Anak),Ya ASI Ekslusif,Ya ada riwayat TB orang serumah,Tidak ada riwayat penyakit diabetes (orang tua),"Asma, Eczema, Asam Urat, Hipertensi, Kolestero...",36 - 54 m^2,3 Kamar,3 Orang,Ada ventilasi namun selalu tertutup,3,0,17.187500
7,84.0,10.00,Laki - laki,Kelurahan Palakka,Kecamatan Barru,Barru,Ayah ASN,Ibu ASN,2.000.000 - 5.000.000,Tidak pernah diabetes,Tidak Anak Telah BCG,Pneumonia (Anak),Tidak ASI Ekslusif,Tidak ada riwayat TB orang serumah,Tidak ada riwayat penyakit diabetes (orang tua),"Hipertensi, Kolesterol (orang serumah)",54 - 120 m^2,3 Kamar,5 Orang,Ada ventilasi namun kadang terbuka/tertutup,2,8,14.172336
8,125.0,27.00,Perempuan,Kelurahan Malimongan Baru,Kecamatan Bontoala,Makassar,Ayah Wiraswasta,Ibu Wiraswasta,> 10.000.000,Tidak pernah diabetes,Ya Anak Telah BCG,"Demam, Radang Tenggorokan (Anak)",Ya ASI Ekslusif,Tidak ada riwayat TB orang serumah,Tidak ada riwayat penyakit diabetes (orang tua),Hipertensi (orang serumah),> 120 m^2,4 Kamar,7 Orang,Setiap ruangan terdapat ventilasi,7,0,17.280000
9,80.0,10.50,Laki - laki,Ke

In [ ]:
cp['Tahun'] = cp['Tahun'].astype(int)
cp['Bulan'] = cp['Bulan'].astype(int)
cp.dtypes

Tinggi badan (dalam cm)               float64
Berat badan (dalam kg)                float64
Jenis Kelamin                          object
Alamat (Kelurahan)                     object
Alamat (Kecamatan)                     object
Alamat (Kab/Kota)                      object
Pekerjaan Ayah                         object
Pekerjaan Ibu                          object
Pendapatan Orang Tua                   object
riwayat diabetes anak                  object
riwayat vaksin BCG                     object
daftar penyakit opname                 object
ASI eksklusif                          object
riwayat TB orang serumah               object
riwayat diabetes keluarga              object
daftar penyakit lain orang serumah     object
luas rumah                             object
jumlah kamar tidur                     object
jumlah orang dalam rumah               object
sistem ventilasi                       object
Tahun                                   int64
Bulan                             

In [ ]:
IMT_laki = pd.read_csv('/content/gdrive/MyDrive/dataset TB Anak/status_gizi_laki.csv', header =1)
IMT_laki.head()

,Tahun,Bulan,-3 SD,-2 SD,-1 SD,Median,+1 SD,+2 SD,+3 SD
0,NaN,0,10.2,11.1,12.2,13.4,14.8,16.3,18.1
1,NaN,1,11.3,12.4,13.6,14.9,16.3,17.8,19.4
2,NaN,2,12.5,13.7,15.0,16.3,17.8,19.4,21.1
3,NaN,3,13.1,14.3,15.5,16.9,18.4,20.0,21.8
4,NaN,4,13.4,14.5,15.8,17.2,18.7,20.3,22.1


In [ ]:
IMT_perempuan = pd.read_csv('/content/gdrive/MyDrive/dataset TB Anak/status_gizi_perempuan.csv', header =1)
IMT_perempuan.head()

,Tahun,Bulan,-3 SD,-2 SD,-1 SD,Median,+1 SD,+2 SD,+3 SD
0,NaN,0,10.1,11.1,12.2,13.3,14.6,16.1,17.7
1,NaN,1,10.8,12.0,13.2,14.6,16.0,17.5,19.1
2,NaN,2,11.8,13.0,14.3,15.8,17.3,19.0,20.7
3,NaN,3,12.4,13.6,14.9,16.4,17.9,19.7,21.5
4,NaN,4,12.7,13.9,15.2,16.7,18.3,20.0,22.0


In [ ]:
IMT_laki.dtypes

Tahun     float64
Bulan       int64
-3 SD     float64
-2 SD     float64
-1 SD     float64
Median    float64
+1 SD     float64
+2 SD     float64
+3 SD     float64
dtype: object

In [ ]:
def count_z_score(jenis_kelamin, tahun, bulan, imt):
  def count_z(idx):
    if (imt > status_gender['Median'][idx[0]]):
      z = (imt - status_gender['Median'][idx[0]]) / (status_gender['+1 SD'][idx[0]] - status_gender['Median'][idx[0]])
    else:
      z = (imt - status_gender['Median'][idx[0]]) / (status_gender['Median'][idx[0]] - status_gender['-1 SD'][idx[0]])
    return z

  if jenis_kelamin == 'Laki - laki':
    status_gender = IMT_laki.copy()
  else:
    status_gender = IMT_perempuan.copy()
  
  if (tahun < 5):
    tahun = tahun * 12
    bulan = tahun + bulan
    index = status_gender.index[status_gender['Bulan'] == bulan].tolist()
    z_score = count_z(index)
  else:
    index = status_gender.index[(status_gender['Tahun'] == tahun) & (status_gender['Bulan'] == bulan)].tolist()
    if ((tahun == 5) & (bulan == 0)):
      index = status_gender.index[(status_gender['Tahun'] == 5) & (status_gender['Bulan'] == 1)].tolist()
      index[0] = index[0] - 1
    z_score = count_z(index)
  return z_score

# z_score = np.round(count_z_score('Perempuan', 10, 0, 19.6), 2)
# print(z_score)
status_gizi = []
i = 0
for imt in cp['IMT']:
  z_score = np.round(count_z_score(cp['Jenis Kelamin'][i], int(cp['Tahun'][i]), int(cp['Bulan'][i]), cp['IMT'][i]), 2)
  
  if (int(cp['Tahun'][i]) < 5):
    if (z_score < -3):
      status = "Gizi buruk"
    elif ((z_score > -3) & (z_score < -2)):
      status = "Gizi kurang"
    elif ((z_score >= -2) & (z_score < 1)):
      status = "Gizi baik"
    elif ((z_score >= 1) & (z_score < 2)):
      status = "Berisiko gizi lebih"
    elif ((z_score >= 2) & (z_score < 3)):
      status = "Gizi lebih"
    elif ((z_score) >= 3):
      status = "Obesitas"
  else:
    if ((z_score >= -3) & (z_score < -2)):
      status = "Gizi kurang"
    elif ((z_score >= -2) & (z_score < 1)):
      status = "Gizi baik"
    elif ((z_score >= 1) & (z_score < 2)):
      status = "Gizi lebih"
    elif ((z_score) >= 2):
      status = "Obesitas"

  print("IMT = ", imt, ", Z score = ", z_score)
  status_gizi.append(status)
  i += 1

cp['Status Gizi'] = status_gizi

IMT =  12.16 , Z score =  -2.62
IMT =  20.833333333333336 , Z score =  1.76
IMT =  15.717455621301774 , Z score =  0.17
IMT =  15.972222222222223 , Z score =  0.14
IMT =  17.28395061728395 , Z score =  1.27
IMT =  12.492563950029746 , Z score =  -3.26
IMT =  17.187499999999996 , Z score =  1.22
IMT =  14.172335600907031 , Z score =  -1.39
IMT =  17.28 , Z score =  0.99
IMT =  16.406249999999996 , Z score =  0.24
IMT =  14.872099940511603 , Z score =  -0.93
IMT =  21.580033777444175 , Z score =  1.3
IMT =  17.465149815734655 , Z score =  1.13
IMT =  15.624999999999996 , Z score =  -0.55
IMT =  17.089180624717066 , Z score =  1.15
IMT =  17.99307958477509 , Z score =  1.76
IMT =  11.406324704596681 , Z score =  -2.68
IMT =  13.536026655560184 , Z score =  -1.55
IMT =  18.367346938775512 , Z score =  2.12
IMT =  16.224986479177936 , Z score =  0.33
IMT =  14.876033057851238 , Z score =  -0.27
IMT =  14.532963403355794 , Z score =  -0.89
IMT =  12.698412698412698 , Z score =  -2.08
IMT =  

In [ ]:
cp.drop(['Tinggi badan (dalam cm)', 'Berat badan (dalam kg)', 'Tahun',
         'Bulan', 'IMT'],
             axis = 'columns', inplace = True)
cp

,Jenis Kelamin,Alamat (Kelurahan),Alamat (Kecamatan),Alamat (Kab/Kota),Pekerjaan Ayah,Pekerjaan Ibu,Pendapatan Orang Tua,riwayat diabetes anak,riwayat vaksin BCG,daftar penyakit opname,ASI eksklusif,riwayat TB orang serumah,riwayat diabetes keluarga,daftar penyakit lain orang serumah,luas rumah,jumlah kamar tidur,jumlah orang dalam rumah,sistem ventilasi,Status Gizi
0,Laki - laki,Kelurahan Noeha,Kecamatan Nuha,Sorowako,Ayah Pegawai Swasta,Ibu Ibu Rumah Tangga,> 10.000.000,Tidak pernah diabetes,Ya Anak Telah BCG,Diare (Anak),Ya ASI Ekslusif,Tidak ada riwayat TB orang serumah,Tidak ada riwayat penyakit diabetes (orang tua),Hipertensi (orang serumah),> 120 m^2,4 Kamar,8 Orang,Setiap ruangan terdapat ventilasi,Gizi kurang
1,Perempuan,Kelurahan Parang Tambung,Kecamatan Tamalate,Makassar,Ayah Wiraswasta,Ibu ASN,2.000.000 - 5.000.000,Tidak pernah diabetes,Tidak Anak Telah BCG,Brongkhitis (Anak),Ya ASI Ekslusif,Tidak ada riwayat TB orang serumah,Tidak ada riwayat penyakit diabetes (orang tua),Tidak Ada (orang serumah),> 120 m^2,5 Kamar,8 Orang,Setiap ruangan terdapat ventilasi,Gizi lebih
2,Laki - laki,Kelurahan Kalebajeng,Kecamatan Bajeng,Gowa,Ayah Wiraswasta,Ibu Ibu rumah tangga,2.000.000 - 5.000.000,Tidak pernah diabetes,Ya Anak Telah BCG,"Tipoid, Diare, Bronkopneumonia, Demam (Anak)",Tidak ASI Ekslusif,Tidak ada riwayat TB orang serumah,Tidak ada riwayat penyakit diabetes (orang tua),Tidak Ada (orang serumah),54 - 120 m^2,2 Kamar,5 Orang,Setiap ruangan terdapat ventilasi,Gizi baik
3,Perempuan,Kelurahan Bone,Kecamatan Masamba,Luwu Utara,Ayah ASN,Ibu ASN,5.000.000 - 10.000.000,Tidak pernah diabetes,Tidak Anak Telah BCG,Tidak Pernah (Anak),Ya ASI Ekslusif,Tidak ada riwayat TB orang serumah,Tidak ada riwayat penyakit diabetes (orang tua),Tidak Ada (orang serumah),54 - 120 m^2,4 Kamar,5 Orang,Setiap ruangan terdapat ventilasi,Gizi baik
4,Perempuan,Kelurahan Borong,Kecamatan Manggala,Makassar,Ayah ASN,Ibu Dokter Gigi,> 10.000.000,Tidak pernah diabetes,Ya Anak Telah BCG,Tuberkulosis (Anak),Tidak ASI Ekslusif,Tidak ada riwayat TB orang serumah,Tidak ada riwayat penyakit diabetes (orang tua),Bronkitis (orang serumah),54 - 120 m^2,4 Kamar,5 Orang,Ada ventilasi namun kadang terbuka/tertutup,Berisiko gizi lebih
5,Laki - laki,Kelurahan Sengka,Kecamatan Bontonompo Selatan,Gowa,Ayah Perawat,Ibu Ibu Rumah tangga,2.000.000 - 5.000.000,Tidak pernah diabetes,Ya Anak Telah BCG,Tidak Pernah (Anak),Ya ASI Ekslusif,Tidak ada riwayat TB orang serumah,Ya ada riwayat penyakit diabetes (orang tua),Tidak ada (orang serumah),36 - 54 m^2,3 Kamar,4 Orang,Ada ventilasi namun kadang terbuka/tertutup,Gizi buruk
6,Laki - laki,Kelurahan Sudiang Raya,Kecamatan Biring Kanaya,Makassar,Ayah Pegawai swasta,Ibu Ibu Rumah tangga,5.000.000 - 10.000.000,Tidak pernah diabetes,Ya Anak Telah BCG,Tidak Pernah (Anak),Ya ASI Ekslusif,Ya ada riwayat TB orang serumah,Tidak ada riwayat penyakit diabetes (orang tua),"Asma, Eczema, Asam Urat, Hipertensi, Kolestero...",36 - 54 m^2,3 Kamar,3 Orang,Ada ventilasi namun selalu tertutup,Berisiko gizi lebih
7,Laki - laki,Kelurahan Palakka,Kecamatan Barru,Barru,Ayah ASN,Ibu ASN,2.000.000 - 5.000.000,Tidak pernah diabetes,Tidak Anak Telah BCG,Pneumonia (Anak),Tidak ASI Ekslusif,Tidak ada riwayat TB orang serumah,Tidak ada riwayat penyakit diabetes (orang tua),"Hipertensi, Kolesterol (orang serumah)",54 - 120 m^2,3 Kamar,5 Orang,Ada ventilasi namun kadang terbuka/tertutup,Gizi baik
8,Perempuan,Kelurahan Malimongan Baru,Kecamatan Bontoala,Makassar,Ayah Wiraswasta,Ibu Wiraswasta,> 10.000.000,Tidak pernah diabetes,Ya Anak Telah BCG,"Demam, Radang Tenggorokan (Anak)",Ya ASI Ekslusif,Tidak ada riwayat TB orang serumah,Tidak ada riwayat penyakit diabetes (orang tua),Hipertensi (orang serumah),> 120 m^2,4 Kamar,7 Orang,Setiap ruangan terdapat ventilasi,Gizi baik
9,Laki - laki,Kelurahan Tamalanrea Jaya,Kecamatan Tamalanrea,Makassar,Ayah Pegawai Swasta,Ibu Tidak Ada,2.000.000 - 5.000.000,Tidak pernah diabetes,Ya Anak Telah BCG,Tidak Pernah (Anak),Ya ASI E

In [ ]:
kecamatan = cp['Alamat (Kecamatan)'].unique()
length = len(kecamatan)
print(kecamatan, length)

['Kecamatan Nuha' 'Kecamatan Tamalate' 'Kecamatan Bajeng'
 'Kecamatan Masamba' 'Kecamatan Manggala' 'Kecamatan Bontonompo Selatan'
 'Kecamatan Biring Kanaya' 'Kecamatan Barru' 'Kecamatan Bontoala'
 'Kecamatan Tamalanrea' 'Kecamatan Rappocini' 'Kecamatan Moncongloe'
 'Kecamatan Sinjai Tengah' 'Kecamatan Panakkukang'
 'Kecamatan Ternate Selatan' 'Kecamatan Mariso' 'Kecamatan Barombong'
 'Kecamatan Biringkanaya' 'Kecamatan Polewali' 'Kecamatan Somba Opu'
 'Kecamatan Bajeng Barat' 'Kecamatan Bantaeng' 'Kecamatan Pattallassang'
 'Kecamatan Labakkang' 'Kecamatan Marusu' 'Kecamatan Bontomarannu'
 'Kecamatan Mangarabombang' 'Kecamatan Cikarang Selatan'] 28


In [ ]:
kelurahan = cp['Alamat (Kelurahan)'].unique()
lengthh = len(kelurahan)
print(kelurahan, lengthh)

['Kelurahan Noeha' 'Kelurahan Parang Tambung' 'Kelurahan Kalebajeng'
 'Kelurahan Bone' 'Kelurahan Borong' 'Kelurahan Sengka'
 'Kelurahan Sudiang Raya' 'Kelurahan Palakka' 'Kelurahan Malimongan Baru'
 'Kelurahan Tamalanrea Jaya' 'Kelurahan Jongaya' 'Kelurahan Karunrung'
 'Kelurahan Moncongloe Lappara' 'Kelurahan Tidung' 'Kelurahan Manggala'
 'Kelurahan Pattongko' 'Kelurahan Panaikang' 'Kelurahan Tamamaung'
 'Kelurahan Tanah Tinggi Barat' 'Kelurahan Mariso'
 'Kelurahan Tamannyeleng' 'Kelurahan Tamalanrea' 'Kelurahan Mataallo'
 'Kelurahan Barua' 'Kelurahan Talasalapang' 'Kelurahan Barombong'
 'Kelurahan Polewali' 'Kelurahan Kalegowa' 'Kelurahan Gunung Sari'
 'Kelurahan Bontomanai' 'Kelurahan Bangkala' 'Kelurahan Batua'
 'Kelurahan Bonto-Bontoa' 'Kelurahan Romangpolong'
 'Kelurahan Pallantikang' 'Kelurahan Sombalabella' 'Kelurahan Mappala'
 'Kelurahan Tanah Tinggi' 'Kelurahan Labakkang' 'Kelurahan Marumpa'
 'Kelurahan Pakatto' 'Kelurahan Romanglompoa' 'Kelurahan Cikoang'
 'Kelurahan Serang

In [ ]:
cp.columns

Index(['Jenis Kelamin', 'Alamat (Kelurahan)', 'Alamat (Kecamatan)',
       'Alamat (Kab/Kota)', 'Pekerjaan Ayah', 'Pekerjaan Ibu',
       'Pendapatan Orang Tua', 'riwayat diabetes anak', 'riwayat vaksin BCG',
       'daftar penyakit opname', 'ASI eksklusif', 'riwayat TB orang serumah',
       'riwayat diabetes keluarga', 'daftar penyakit lain orang serumah',
       'luas rumah', 'jumlah kamar tidur', 'jumlah orang dalam rumah',
       'sistem ventilasi', 'Status Gizi'],
      dtype='object')

In [ ]:
cp["variabel"] = cp[['Jenis Kelamin', 'Alamat (Kelurahan)', 'Alamat (Kecamatan)',
       'Alamat (Kab/Kota)', 'Pekerjaan Ayah', 'Pekerjaan Ibu',
       'Pendapatan Orang Tua', 'riwayat diabetes anak', 'riwayat vaksin BCG',
       'daftar penyakit opname', 'ASI eksklusif', 'riwayat TB orang serumah',
       'riwayat diabetes keluarga', 'daftar penyakit lain orang serumah',
       'luas rumah', 'jumlah kamar tidur', 'jumlah orang dalam rumah',
       'sistem ventilasi', 'Status Gizi']].agg(','.join, axis=1)
       
print(cp['variabel'])

0     Laki - laki,Kelurahan Noeha,Kecamatan Nuha,Sor...
1     Perempuan,Kelurahan Parang Tambung,Kecamatan T...
2     Laki - laki,Kelurahan Kalebajeng,Kecamatan Baj...
3     Perempuan,Kelurahan Bone,Kecamatan Masamba,Luw...
4     Perempuan,Kelurahan Borong,Kecamatan Manggala,...
5     Laki - laki,Kelurahan Sengka,Kecamatan Bontono...
6     Laki - laki,Kelurahan Sudiang Raya,Kecamatan B...
7     Laki - laki,Kelurahan Palakka,Kecamatan Barru,...
8     Perempuan,Kelurahan Malimongan Baru,Kecamatan ...
9     Laki - laki,Kelurahan Tamalanrea Jaya,Kecamata...
10    Laki - laki,Kelurahan Jongaya,Kecamatan Tamala...
11    Laki - laki,Kelurahan Parang Tambung,Kecamatan...
12    Laki - laki,Kelurahan Karunrung,Kecamatan Rapp...
13    Perempuan,Kelurahan Moncongloe Lappara,Kecamat...
14    Laki - laki,Kelurahan Tidung,Kecamatan Rappoci...
15    Laki - laki,Kelurahan Manggala,Kecamatan Mangg...
16    Perempuan,Kelurahan Pattongko,Kecamatan Sinjai...
17    Perempuan,Kelurahan Panaikang,Kecamatan Pa

In [ ]:
cp_array = cp["variabel"].apply(lambda x: x.split(','))
print(cp_array.to_numpy)

<bound method IndexOpsMixin.to_numpy of 0     [Laki - laki, Kelurahan Noeha, Kecamatan Nuha,...
1     [Perempuan, Kelurahan Parang Tambung, Kecamata...
2     [Laki - laki, Kelurahan Kalebajeng, Kecamatan ...
3     [Perempuan, Kelurahan Bone, Kecamatan Masamba,...
4     [Perempuan, Kelurahan Borong, Kecamatan Mangga...
5     [Laki - laki, Kelurahan Sengka, Kecamatan Bont...
6     [Laki - laki, Kelurahan Sudiang Raya, Kecamata...
7     [Laki - laki, Kelurahan Palakka, Kecamatan Bar...
8     [Perempuan, Kelurahan Malimongan Baru, Kecamat...
9     [Laki - laki, Kelurahan Tamalanrea Jaya, Kecam...
10    [Laki - laki, Kelurahan Jongaya, Kecamatan Tam...
11    [Laki - laki, Kelurahan Parang Tambung, Kecama...
12    [Laki - laki, Kelurahan Karunrung, Kecamatan R...
13    [Perempuan, Kelurahan Moncongloe Lappara, Keca...
14    [Laki - laki, Kelurahan Tidung, Kecamatan Rapp...
15    [Laki - laki, Kelurahan Manggala, Kecamatan Ma...
16    [Perempuan, Kelurahan Pattongko, Kecamatan Sin...
17    [P

In [ ]:
# cp_str = cp_array.astype('str').apply(lambda x: x.split(','))
# cp_str

In [ ]:
test_data = [['I1','I2','I5'],
             ['I2','I4'],
             ['I2','I3'],
             ['I1','I2','I4'],
             ['I1','I3'],
             ['I2','I3'],
             ['I1','I3'],
             ['I1','I2','I3','I5'],
             ['I1','I2','I3']]


In [ ]:
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_ary = te.fit(cp_array).transform(cp_array)
df = pd.DataFrame(te_ary, columns=te.columns_)
df

In [ ]:
import numpy as np
import pandas as pd
import collections

In [ ]:
def setup_fptree(df, min_support):
    num_itemsets = len(df.index)        # number of itemsets in the database

    is_sparse = False
    if hasattr(df, "sparse"):
        # DataFrame with SparseArray (pandas >= 0.24)
        if df.size == 0:
            itemsets = df.values
        else:
            itemsets = df.sparse.to_coo().tocsr()
            is_sparse = True
    else:
        # dense DataFrame
        itemsets = df.values

    # support of each individual item
    # if itemsets is sparse, np.sum returns an np.matrix of shape (1, N)
    item_support = np.array(np.sum(itemsets, axis=0) / float(num_itemsets))
    item_support = item_support.reshape(-1)

    items = np.nonzero(item_support >= min_support)[0]

    # Define ordering on items for inserting into FPTree
    indices = item_support[items].argsort()
    rank = {item: i for i, item in enumerate(items[indices])}

    if is_sparse:
        # Ensure that there are no zeros in sparse DataFrame
        itemsets.eliminate_zeros()

    # Building tree by inserting itemsets in sorted order
    # Heuristic for reducing tree size is inserting in order
    #   of most frequent to least frequent
    tree = FPTree(rank)
    for i in range(num_itemsets):
        if is_sparse:
            # itemsets has been converted to CSR format to speed-up the line
            # below.  It has 3 attributes:
            #  - itemsets.data contains non null values, shape(#nnz,)
            #  - itemsets.indices contains the column number of non null
            #    elements, shape(#nnz,)
            #  - itemsets.indptr[i] contains the offset in itemset.indices of
            #    the first non null element in row i, shape(1+#nrows,)
            nonnull = itemsets.indices[itemsets.indptr[i]:itemsets.indptr[i+1]]
        else:
            nonnull = np.where(itemsets[i, :])[0]
        itemset = [item for item in nonnull if item in rank]
        itemset.sort(key=rank.get, reverse=True)
        tree.insert_itemset(itemset)

    return tree, rank

In [ ]:
def generate_itemsets(generator, num_itemsets, colname_map):
    itemsets = []
    supports = []
    for sup, iset in generator:
        itemsets.append(frozenset(iset))
        supports.append(sup / num_itemsets)

    res_df = pd.DataFrame({'support': supports, 'itemsets': itemsets})

    if colname_map is not None:
        res_df['itemsets'] = res_df['itemsets'] \
            .apply(lambda x: frozenset([colname_map[i] for i in x]))

    return res_df

In [ ]:
def valid_input_check(df):

    if f"{type(df)}" == "<class 'pandas.core.frame.SparseDataFrame'>":
        msg = ("SparseDataFrame support has been deprecated in pandas 1.0,"
               " and is no longer supported in mlxtend. "
               " Please"
               " see the pandas migration guide at"
               " https://pandas.pydata.org/pandas-docs/"
               "stable/user_guide/sparse.html#sparse-data-structures"
               " for supporting sparse data in DataFrames.")
        raise TypeError(msg)

    if df.size == 0:
        return
    if hasattr(df, "sparse"):
        if not isinstance(df.columns[0], str) and df.columns[0] != 0:
            raise ValueError('Due to current limitations in Pandas, '
                             'if the sparse format has integer column names,'
                             'names, please make sure they either start '
                             'with `0` or cast them as string column names: '
                             '`df.columns = [str(i) for i in df.columns`].')

    # Fast path: if all columns are boolean, there is nothing to checks
    all_bools = df.dtypes.apply(pd.api.types.is_bool_dtype).all()
    if not all_bools:
        # Pandas is much slower than numpy, so use np.where on Numpy arrays
        if hasattr(df, "sparse"):
            if df.size == 0:
                values = df.values
            else:
                values = df.sparse.to_coo().tocoo().data
        else:
            values = df.values
        idxs = np.where((values != 1) & (values != 0))
        if len(idxs[0]) > 0:
            # idxs has 1 dimension with sparse data and 2 with dense data
            val = values[tuple(loc[0] for loc in idxs)]
            s = ('The allowed values for a DataFrame'
                 ' are True, False, 0, 1. Found value %s' % (val))
            raise ValueError(s)

In [ ]:
class FPTree(object):
    def __init__(self, rank=None):
        self.root = FPNode(None)
        self.nodes = collections.defaultdict(list)
        self.cond_items = []
        self.rank = rank

    def conditional_tree(self, cond_item, minsup):
        """
        Creates and returns the subtree of self conditioned on cond_item.
        Parameters
        ----------
        cond_item : int | str
            Item that the tree (self) will be conditioned on.
        minsup : int
            Minimum support threshold.
        Returns
        -------
        cond_tree : FPtree
        """
        # Find all path from root node to nodes for item
        branches = []
        count = collections.defaultdict(int)
        for node in self.nodes[cond_item]:
            branch = node.itempath_from_root()
            branches.append(branch)
            for item in branch:
                count[item] += node.count

        # Define new ordering or deep trees may have combinatorially explosion
        items = [item for item in count if count[item] >= minsup]
        items.sort(key=count.get)
        rank = {item: i for i, item in enumerate(items)}

        # Create conditional tree
        cond_tree = FPTree(rank)
        for idx, branch in enumerate(branches):
            branch = sorted([i for i in branch if i in rank],
                            key=rank.get, reverse=True)
            cond_tree.insert_itemset(branch, self.nodes[cond_item][idx].count)
        cond_tree.cond_items = self.cond_items + [cond_item]

        return cond_tree

    def insert_itemset(self, itemset, count=1):
        """
        Inserts a list of items into the tree.
        Parameters
        ----------
        itemset : list
            Items that will be inserted into the tree.
        count : int
            The number of occurrences of the itemset.
        """
        self.root.count += count

        if len(itemset) == 0:
            return

        # Follow existing path in tree as long as possible
        index = 0
        node = self.root
        for item in itemset:
            if item in node.children:
                child = node.children[item]
                child.count += count
                node = child
                index += 1
            else:
                break

        # Insert any remaining items
        for item in itemset[index:]:
            child_node = FPNode(item, count, node)
            self.nodes[item].append(child_node)
            node = child_node

    def is_path(self):
        if len(self.root.children) > 1:
            return False
        for i in self.nodes:
            if len(self.nodes[i]) > 1 or len(self.nodes[i][0].children) > 1:
                return False
        return True

    def print_status(self, count, colnames):
        cond_items = [str(i) for i in self.cond_items]
        if colnames:
            cond_items = [str(colnames[i]) for i in self.cond_items]
        cond_items = ", ".join(cond_items)
        print('\r%d itemset(s) from tree conditioned on items (%s)' %
              (count, cond_items), end="\n")


In [ ]:
class FPNode(object):
    def __init__(self, item, count=0, parent=None):
        self.item = item
        self.count = count
        self.parent = parent
        self.children = collections.defaultdict(FPNode)

        if parent is not None:
            parent.children[item] = self

    def itempath_from_root(self):
        """ Returns the top-down sequence of items from self to
            (but not including) the root node. """
        path = []
        if self.item is None:
            return path

        node = self.parent
        while node.item is not None:
            path.append(node.item)
            node = node.parent

        path.reverse()
        return path

In [ ]:
import math
import itertools

In [ ]:
def fpgrowth(df, min_support=0.1, use_colnames=False, max_len=None, verbose=0):
    if min_support <= 0.:
        raise ValueError('`min_support` must be a positive '
                         'number within the interval `(0, 1]`. '
                         'Got %s.' % min_support)

    colname_map = None
    if use_colnames:
        colname_map = {idx: item for idx, item in enumerate(df.columns)}

    tree, _ = setup_fptree(df, min_support)
    minsup = math.ceil(min_support * len(df.index))  # min support as count
    generator = fpg_step(tree, minsup, colname_map, max_len, verbose)

    return generate_itemsets(generator, len(df.index), colname_map)


In [ ]:
def fpg_step(tree, minsup, colnames, max_len, verbose):
    """
    Performs a recursive step of the fpgrowth algorithm.
    Parameters
    ----------
    tree : FPTree
    minsup : int
    Yields
    ------
    lists of strings
        Set of items that has occurred in minsup itemsets.
    """
    count = 0
    items = tree.nodes.keys()
    if tree.is_path():
        # If the tree is a path, we can combinatorally generate all
        # remaining itemsets without generating additional conditional trees
        size_remain = len(items) + 1
        if max_len:
            size_remain = max_len - len(tree.cond_items) + 1
        for i in range(1, size_remain):
            for itemset in itertools.combinations(items, i):
                count += 1
                support = min([tree.nodes[i][0].count for i in itemset])
                yield support, tree.cond_items + list(itemset)
    elif not max_len or max_len > len(tree.cond_items):
        for item in items:
            count += 1
            support = sum([node.count for node in tree.nodes[item]])
            yield support, tree.cond_items + [item]

    if verbose:
        tree.print_status(count, colnames)

    # Generate conditional trees to generate frequent itemsets one item larger
    if not tree.is_path() and (not max_len or max_len > len(tree.cond_items)):
        for item in items:
            cond_tree = tree.conditional_tree(item, minsup)
            for sup, iset in fpg_step(cond_tree, minsup,
                                      colnames, max_len, verbose):
                yield sup, iset

In [ ]:
import pprint

In [ ]:
fpp = fpgrowth(df, min_support=0.5, use_colnames= True)
pprint.pprint(fpp)

In [ ]:
fpp.shape

In [ ]:
def conviction(sAC, sA, sC):
  confidence = sAC/sA
  conviction = np.empty(confidence.shape, dtype=float)
  if not len(conviction.shape):
      conviction = conviction[np.newaxis]
      confidence = confidence[np.newaxis]
      sAC = sAC[np.newaxis]
      sA = sA[np.newaxis]
      sC = sC[np.newaxis]
  conviction[:] = np.inf
  conviction[confidence < 1.] = ((1. - sC[confidence < 1.]) /
                                  (1. - confidence[confidence < 1.]))

  return conviction

In [ ]:
def get_rules(res_df, metric = "confidence", min_threshold = 0.8, support_only = False):
  if not res_df.shape[0]:
        raise ValueError('The input DataFrame `df` containing '
                         'the frequent itemsets is empty.')
        
  if not all(col in res_df.columns for col in ["support", "itemsets"]):
        raise ValueError("Dataframe needs to contain the\
                         columns 'support' and 'itemsets'")
        
  metric_dict = {
        "antecedent support": lambda _, sA, __: sA,
        "consequent support": lambda _, __, sC: sC,
        "support": lambda sAC, _, __: sAC,
        "confidence": lambda sAC, sA, _: sAC/sA,
        "lift": lambda sAC, sA, sC: metric_dict["confidence"](sAC, sA, sC)/sC,
        "leverage": lambda sAC, sA, sC: metric_dict["support"](
             sAC, sA, sC) - sA*sC,
        "conviction": lambda sAC, sA, sC: conviction(sAC, sA, sC)
        }

  columns_ordered = ["antecedent support",
                   "consequent support",
                   "support",
                   "confidence",
                   "lift",
                   "leverage", 
                   "conviction"]

  if support_only:
    metric = 'support'
  else:
    if metric not in metric_dict.keys():
        raise ValueError("Metric must be 'confidence' or 'lift', got '{}'"
                          .format(metric))

  # get dict of {frequent itemset} -> support
  keys = res_df['itemsets'].values
  values = res_df['support'].values
  frozenset_vect = np.vectorize(lambda x: frozenset(x))
  frequent_items_dict = dict(zip(frozenset_vect(keys), values))

  # prepare buckets to collect frequent rules
  rule_antecedents = []
  rule_consequents = []
  rule_supports = []

  for k in frequent_items_dict.keys():
    sAC = frequent_items_dict[k]
    # to find all possible combinations
    for idx in range(len(k)-1, 0, -1):
      # of antecedent and consequent
      for c in combinations(k, r=idx):
        antecedent = frozenset(c)
        consequent = k.difference(antecedent)

        if support_only:
          # support doesn't need these,
          # hence, placeholders should suffice
          sA = None
          sC = None

        else:
          try:
            sA = frequent_items_dict[antecedent]
            sC = frequent_items_dict[consequent]
          except KeyError as e:
            s = (str(e) + 'You are likely getting this error'
                          ' because the DataFrame is missing '
                          ' antecedent and/or consequent '
                          ' information.'
                          ' You can try using the '
                          ' `support_only=True` option')
            raise KeyError(s)
          # check for the threshold

        score = metric_dict[metric](sAC, sA, sC)
        if score >= min_threshold:
            rule_antecedents.append(antecedent)
            rule_consequents.append(consequent)
            rule_supports.append([sAC, sA, sC])

  # check if frequent rule was generated
  if not rule_supports:
    return pd.DataFrame(
      columns=["antecedents", "consequents"] + columns_ordered)

  else:
    # generate metrics
    rule_supports = np.array(rule_supports).T.astype(float)
    df_res = pd.DataFrame(
      data=list(zip(rule_antecedents, rule_consequents)),
      columns=["antecedents", "consequents"])

    if support_only:
      sAC = rule_supports[0]
      for m in columns_ordered:
          df_res[m] = np.nan
      df_res['support'] = sAC

    else:
      sAC = rule_supports[0]
      sA = rule_supports[1]
      sC = rule_supports[2]
      for m in columns_ordered:
        df_res[m] = metric_dict[m](sAC, sA, sC)

    return df_res

In [ ]:
from itertools import combinations

In [ ]:
pd.set_option('display.max_rows', None)
rules = get_rules(fpp)
rules